In [ ]:
!git clone https://github.com/openai/improved-diffusion.git
%cd improved-diffusion
!pip install -e .


In [2]:
pip install mpi4py


  Using cached mpi4py-4.1.1-cp312-cp312-manylinux1_x86_64.manylinux_2_5_x86_64.whl.metadata (16 kB)
Using cached mpi4py-4.1.1-cp312-cp312-manylinux1_x86_64.manylinux_2_5_x86_64.whl (1.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

In [4]:
import sys
print(sys.executable)


/opt/conda/bin/python


In [5]:
!conda install -c conda-forge openmpi -y
!conda install -c conda-forge mpi4py -y


Channels:
 - conda-forge
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.5.1
    latest version: 25.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - openmpi


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    attr-2.5.2                 |       h39aace5_0          66 KB  conda-forge
    ca-certificates-2025.11.12 |       hbd8a1cb_0         149 KB  conda-forge
    certifi-2025.11.12         |     pyhd8ed1ab_0         153 KB  conda-forge
    libcap-2.77                |       h3ff7636_0         119 KB  conda-forge
    libfabric-2.4.0            |       ha770c72_0          14 KB  conda-forge
    libfabric1-2.4.0           |       h6c8fc0a_0         686 KB  conda-forge
    libhwloc-2.12.1          

In [5]:
! /opt/conda/bin/python -m pip install --upgrade pip
! /opt/conda/bin/python -m pip install mpi4py


In [6]:
import mpi4py
from mpi4py import MPI

print("MPI version:", MPI.Get_version())
print("MPI rank:", MPI.COMM_WORLD.Get_rank())


MPI version: (3, 1)
MPI rank: 0


In [7]:
RESULTS_DIR = "./DDIMResults" 

os.makedirs(RESULTS_DIR, exist_ok=True)

In [8]:

os.environ["OPENAI_LOGDIR"] = RESULTS_DIR


In [9]:
DATA_DIR = "./combined_skeletons"

In [10]:
MODEL_FLAGS="--image_size 256 --num_channels 128 --num_res_blocks 2 --learn_sigma True"
DIFFUSION_FLAGS="--diffusion_steps 1000 --noise_schedule linear"
TRAIN_FLAGS="--lr 1e-4 --batch_size 16 --microbatch 4"


In [ ]:
!python improved-diffusion/scripts/image_train.py \
    --data_dir $DATA_DIR \
    $MODEL_FLAGS \
    $DIFFUSION_FLAGS \
    $TRAIN_FLAGS


In [11]:
%cd improved-diffusion

/home/jovyan/improved-diffusion


In [12]:
!pip install -e .


Obtaining file:///home/jovyan/improved-diffusion
  Preparing metadata (setup.py) ... done
  Using cached blobfile-3.1.0-py3-none-any.whl.metadata (15 kB)
  Using cached torch-2.9.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached pycryptodomex-3.23.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached lxml-6.0.2-cp312-cp312-manylinux_2_26_x86_64.manylinux_2_28_x86_64.whl.metadata (3.6 kB)
  Using cached filelock-3.20.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cub

In [13]:
%cd ..

/home/jovyan


In [ ]:
!python improved-diffusion/scripts/image_train.py \
  --data_dir $DATA_DIR \
  --resume_checkpoint ./DDIMResults/model030000.pt \
  $MODEL_FLAGS \
  $DIFFUSION_FLAGS \
  $TRAIN_FLAGS


In [14]:
!python improved-diffusion/scripts/image_sample.py \
  --model_path ./DDIMResults/ema_0.9999_090000.pt \
  $MODEL_FLAGS \
  $DIFFUSION_FLAGS \
  --use_ddim False \
  --num_samples 1000 \
  --timestep_respacing 1000


Logging to ./DDIMResults
creating model and diffusion...
sampling...
created 16 samples
created 32 samples
created 48 samples
created 64 samples
created 80 samples
created 96 samples
created 112 samples
created 128 samples
created 144 samples
created 160 samples
created 176 samples
created 192 samples
created 208 samples
created 224 samples
created 240 samples
created 256 samples
created 272 samples
created 288 samples
created 304 samples
created 320 samples
created 336 samples
created 352 samples
created 368 samples
created 384 samples
created 400 samples
created 416 samples
created 432 samples
created 448 samples
created 464 samples
created 480 samples
created 496 samples
created 512 samples
created 528 samples
created 544 samples
created 560 samples
created 592 samples
created 608 samples
created 624 samples
created 640 samples
created 656 samples
created 672 samples
created 688 samples
created 704 samples
created 720 samples
created 736 samples
created 752 samples
created 768 sampl

In [15]:
import os
import numpy as np
from PIL import Image

NPZ_PATH = "./DDIMResults/samples_1000x256x256x3.npz"
OUT_DIR = "./diffusion_samples"

os.makedirs(OUT_DIR, exist_ok=True)

images = np.load(NPZ_PATH)["arr_0"]

print(images.shape, images.dtype)

for i, img in enumerate(images):
    # img is already uint8 [0,255], shape (256,256,3)
    img_gray = Image.fromarray(img).convert("L")
    img_gray.save(os.path.join(OUT_DIR, f"skeleton_{i:06d}.png"))

print(f"Saved {len(images)} grayscale skeleton images")


(1000, 256, 256, 3) uint8
Saved 1000 grayscale skeleton images
